# Emerging Technologies Assessment

In [1]:
import random
from itertools import product

## Problem 1

In [2]:
def random_constant_balanced():
    """
    Returns a randomly chosen function that is either constant or balanced.
    
    The returned function takes four Boolean arguments as inputs and returns
    a single Boolean output.
    
    Returns:
        function: A function f(a, b, c, d) -> bool that is either constant
                  (always returns the same value) or balanced (returns True
                  for exactly half of the 16 possible input combinations).
    """
    # Randomly decide whether to create a constant or balanced function.
    # random.choice() selects one element uniformly at random from the list.
    # See: https://docs.python.org/3/library/random.html#random.choice
    function_type = random.choice(['constant', 'balanced'])
    
    # Check if we should create a constant function.
    if function_type == 'constant':
        # Randomly choose to return always True or always False.
        # This gives equal probability to both constant functions.
        constant_value = random.choice([True, False])
        
        # Define the constant function as a closure.
        def constant_function(a, b, c, d):
            """Constant function that always returns the same value."""
            # Return the predetermined constant value.
            return constant_value
        
        # Return the constant function.
        return constant_function
    
    # Otherwise, create a balanced function.
    else:  # balanced
        # Generate all 16 possible input combinations.
        # product([False, True], repeat=4) creates all 2^4 binary combinations.
        # See: https://docs.python.org/3/library/itertools.html#itertools.product
        all_inputs = list(product([False, True], repeat=4))
        
        # Randomly select 8 combinations that will return True.
        # random.sample() ensures no duplicates and uniform distribution.
        # See: https://docs.python.org/3/library/random.html#random.sample
        # Convert to set for O(1) membership testing later.
        true_inputs = set(random.sample(all_inputs, 8))
        
        # Define the balanced function as a closure.
        def balanced_function(a, b, c, d):
            """Balanced function that returns True for exactly half of inputs."""
            # Check if the input tuple is in the set of inputs that return True.
            # Set membership testing is O(1) on average.
            return (a, b, c, d) in true_inputs
        
        # Return the balanced function.
        return balanced_function

## Problem 2

## Problem 3

## Problem 4

## Problem 5